In [ ]:
import pandas as pd
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.analysis import plotting, plotting_util, pairs_plotting, prim
from ema_workbench import (RealParameter, ScalarOutcome, Constant,
                           Model, MultiprocessingEvaluator, SequentialEvaluator, ema_logging,
                           perform_experiments, Policy, Scenario)
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)
from ema_workbench import Constraint
from ema_workbench.util.utilities import (save_results, load_results)
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# get results from MORO
result_MORO = pd.read_csv('ANALYSIS_results/03_MORO_results.csv')

In [ ]:
# get policies
policies = result_MORO[['0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0', '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0', '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2', 'EWS_DaysToThreat', 'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2', 'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2', 'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2', 'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2', 'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2']]
policies_list = [Policy(str(index), **row.to_dict()) for index, row in policies.iterrows()]

In [ ]:
# define problem formulation
model, functions = get_model_for_problem_formulation(2)

In [ ]:
# run model using sobol sampling
ema_logging.log_to_stderr(ema_logging.INFO)


with MultiprocessingEvaluator(model) as evaluator:
    experiment, result = evaluator.perform_experiments(
                                         scenarios=1000,
                                         policies = policies_list, 
                                         uncertainty_sampling='sobol')

In [ ]:
# save results
results = experiment, result
save_results(results, 'ANALYSIS_results/04_sobol_results.tar.gz')

In [ ]:
# load results
experiment, result = load_results('ANALYSIS_results/04_sobol_results.tar.gz')

In [ ]:
# speficy SA lib problem
SA_lib_problem = get_SALib_problem(model.uncertainties)

In [ ]:
experiment['Minimum water level full network'] = result['Minimum water level full network'] 

In [ ]:
# SOBOL results for first policy
Si = sobol.analyze(SA_lib_problem, experiment[experiment['policy'] == 0]['Minimum water level full network'].to_numpy())
scores_filtered = {k:Si[k] for k in ['ST', 'ST_conf', 'S1', 'S1_conf']}
Si_df_1 = pd.DataFrame(scores_filtered, index= SA_lib_problem['names'])
fig, ax = plt.subplots(1)
indices = Si_df_1[['S1', 'ST']]
err = Si_df_1[['S1_conf', 'ST_conf']]
indices.plot.bar(yerr=err.values.T, ax=ax)
fig.set_size_inches(8,6)
fig.subplots_adjust(bottom=0.3)